# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
spark

## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [2]:
apps = spark.read.csv('Datasets/googleplaystore.csv', inferSchema=True, header=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [3]:
apps.limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


As well as the schema to make sure all the column types were correctly infered

In [4]:
apps.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [26]:
apps.count()

10841

Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [5]:
from pyspark.sql.types import IntegerType, FloatType
apps_new = apps.withColumn("Rating", apps["Rating"].cast(FloatType())) \
            .withColumn("Reviews", apps["Reviews"].cast(IntegerType())) \
            .withColumn("Price", apps["Price"].cast(IntegerType()))
print(apps_new.printSchema())
apps_new.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [6]:
apps_new.createOrReplaceTempView("appsView")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [9]:
spark.sql("SELECT * FROM appsView WHERE Rating > 4.1").limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
1,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
3,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
4,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [17]:
df = spark.sql("SELECT App, Rating, Category FROM appsView WHERE Category = 'COMICS' AND Rating > 4.5")
df.limit(5).toPandas()

,App,Rating,Category
0,Manga Master - Best manga & comic reader,4.6,COMICS
1,GANMA! - All original stories free of charge f...,4.7,COMICS
2,Röhrich Werner Soundboard,4.7,COMICS
3,Unicorn Pokez - Color By Number,4.8,COMICS
4,Manga - read Thai translation,4.6,COMICS


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [38]:
from pyspark.sql.functions import *
spark.sql("SELECT Category, sum(Reviews) AS Total_Reviews FROM appsView GROUP BY Category ORDER By Total_Reviews DESC")\
    .limit(1).toPandas()

,Category,Total_Reviews
0,GAME,1585422349


## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [23]:
spark.sql("SELECT App, Reviews AS Total_Reviews FROM appsView ORDER BY Reviews desc").limit(1).toPandas()

,App,Total_Reviews
0,Facebook,78158306


## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [25]:
spark.sql("SELECT App, Rating FROM appsView WHERE App LIKE '%dating%'").toPandas()

,App,Rating
0,"Meet, chat & date. Free dating app - Chocolate...",3.9
1,Friend Find: free chat + flirt dating app,NaN
2,Spine- The dating app,5.0
3,Princess Closet : Otome games free dating sim,4.5
4,happn – Local dating app,4.3


## 6. Use SQL Transformer to display how many free apps there are in this list

In [29]:
from pyspark.ml.feature import SQLTransformer
sqlTrans = SQLTransformer(statement = "SELECT count(*) as Free_Apps FROM __THIS__ WHERE Type = 'Free'")
sqlTrans.transform(apps_new).show()

+---------+
|Free_Apps|
+---------+
|    10037|
+---------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [33]:
spark.sql("SELECT Genres, count(*) AS count FROM appsView GROUP BY Genres ORDER BY count desc").show(1)

+------+-----+
|Genres|count|
+------+-----+
| Tools|  842|
+------+-----+
only showing top 1 row



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [36]:
spark.sql("SELECT App, Reviews FROM appsView WHERE Genres = 'Tools' AND Reviews > 100").limit(10).toPandas()

,App,Reviews
0,Moto File Manager,38655
1,Google,8033493
2,Google Translate,5745093
3,Moto Display,18239
4,Motorola Alert,24199
5,Motorola Assist,37333
6,Cache Cleaner-DU Speed Booster (booster & clea...,12759663
7,Moto Suggestions ™,308
8,Moto Voice,33216
9,Calculator,40770


## That's all folks! Great job!